# Optymalizacja hiperparametrów 

#### Grid Search 

Na przykładzie modelu *XGBoost* widzimy, że niektóre algorytmy są wysoce parametryzowalne, i prawidłowy dobór hiperparametrów potrafi czasami istotnie podnieść jakość naszych predykcji.  
Poznaliśmy już przeszukiwanie po ustalonej siatce parametrów - `GridSearchCV`. Metoda ta ma jednak dwie dość istotne wady:
* rozmiar siatki bardzo szybko rośnie wraz z ilością parametrów. Już dla czterech parametrów, po 5 wartości każdy, dostajemy $5^4 = 625$ kombinacji. W sytuacji, kiedy model uczy nam się minutę (a to w praktyce bardzo optymisyczny przypadek) musimy czekać 10 godzin na przejście pętli.
* niska gęstość siatki powoduje, że możemy nie wstrzelić się w tą optymalną wartość. Przykład:

 https://cdn-images-1.medium.com/max/1600/1*ZTlQm_WRcrNqL-nLnx6GJA.png


#### Random Search 

Częściowym rozwiązaniem tego problemu jest `RandomizedSearch`, czyli losowe przeszukiwanie przestrzeni (zobrazowane na powyższym przykładzie). 
Zalety takiego rozwiązania:
* wspomniane wyżej dokładniejsze przeszukanie każdego z parametrów
* możliwość zadania dowolnego rozkładu (w praktyce dowolnego zaimplementowanego w scipy) dla zmiennych ciągłych
* szybszy czas działania - dzięki temu że przeszukujemy przestrzeń dokładniej, możemy przez to mocno ograniczyć ilość iteracji.  


Jak podawać wartości parametrów do przetestowania? Możemy to zrobić na dwa sposoby:
* lista wartości - wtedy algorytm po prostu będzie losował jedną z wartości z listy
* ciągły rozkład - losujemy wtedy dowolną wartość z tego rozkładu. w `RandomizedSearchCV` przez rozkład rozumiemy obiekt, który posiada metodę `rvs` zwracającą próbkę z tego rozkładu. Dużo rozkładów znajdziemy w module `scipy.stats.distributions` (https://docs.scipy.org/doc/scipy/reference/stats.html)

In [9]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import seaborn as sns 

In [8]:
import pandas as pd
from sklearn.model_selection import cross_val_score, KFold
from sklearn.metrics import accuracy_score

from sklearn import datasets
cancer = datasets.load_breast_cancer()

X = cancer.data
y = cancer.target

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [13]:
# RandomizedSearchCV??
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt

model = GradientBoostingClassifier()
parameters = {"learning_rate": sp_randFloat(),
                  "subsample"    : sp_randFloat(),
                  "n_estimators" : sp_randInt(100, 1000),
                  "max_depth"    : sp_randInt(4, 10)
                 }


randm = RandomizedSearchCV(estimator=model, param_distributions = parameters,
                               cv = 2, n_iter = 10, n_jobs=-1)
randm.fit(X_train, y_train)

print(" Results from Random Search " )
print(" The best estimator across ALL searched params:", randm.best_estimator_)
print("The best score across ALL searched params:", randm.best_score_)
print("The best parameters across ALL searched params:", randm.best_params_)

 Results from Random Search 
 The best estimator across ALL searched params: GradientBoostingClassifier(learning_rate=0.34892572350985473, max_depth=5,
                           n_estimators=446, subsample=0.7928975705699531)
The best score across ALL searched params: 0.960093896713615
The best parameters across ALL searched params: {'learning_rate': 0.34892572350985473, 'max_depth': 5, 'n_estimators': 446, 'subsample': 0.7928975705699531}


In [12]:
# GridSearchCV??
from sklearn.linear_model import LogisticRegression
grid_values = {'penalty': ['l1', 'l2'],'C':[0.001, 0.10, 0.1, 10, 25, 50, 100, 1000]}
clf = LogisticRegression()
grid = GridSearchCV(clf, grid_values, cv = 10, scoring='accuracy')
grid.fit(X_train, y_train) 
print(grid.best_params_) 

c:\Program Files\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Program Files\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_i

{'C': 25, 'penalty': 'l2'}


c:\Program Files\Python310\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
80 fits failed out of a total of 160.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
80 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Program Files\Python310\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Program Files\Python310\lib\site-packages\sklearn\linear_model\_logistic.py", line 1461, in fit
    solver = _check_solver(self.solver, self.penalty, self.dual)
  File "c:\Program Files\Python310\lib\site-packages\sklearn\linear_model\_logistic.py", line 447, in _check_solver
    raise ValueEr

In [5]:
param_grid = {
    'max_depth': [3, 5, 8, 10],
    'learning_rate': [0.001, 0.01, 0.05, 0.1],
    'n_estimators': [50, 100, 150, 200, 400],
    'gamma': [0, 0.5, 1, 2],
    'colsample_bytree': [1, 0.8, 0.5],
    'subsample': [1, 0.8, 0.5],
    'min_child_weight': [1, 5, 10]
}

In [6]:
from scipy.stats.distributions import uniform, randint

In [7]:
# ??uniform

Rozkłady przyjmują 2 parametry: `loc`, `scale`.

`[2, 3, 4, 5] --> uniform(2, 3)`

In [8]:
uniform_23 = uniform(2, 3)

In [9]:
# ??randint

In [10]:
# randint(low, high) losuje od low do high-1 włącznie!
randint_25 = randint(2, 6)

In [11]:
param_grid = {
    'max_depth': [3, 5, 8, 10],
    'learning_rate': [0.001, 0.01, 0.05, 0.1],
    'n_estimators': [50, 100, 150, 200, 400],
    'gamma': [0, 0.5, 1, 2],
    'colsample_bytree': [1, 0.8, 0.5],
    'subsample': [1, 0.8, 0.5],
    'min_child_weight': [1, 5, 10]
}

param_distribution = {
    'max_depth': randint(3, 11),
    'learning_rate': uniform(0.001, 0.1-0.001),
    'n_estimators': randint(50, 400),
    'gamma': uniform(0,2),
    'colsample_bytree': uniform(0.5, 0.5),
    'subsample': uniform(0.5, 0.5),
    'min_child_weight': randint(1, 11)
}

# Zad.

Zbudować 
* `RandomSearchCV` 
* `GridSearchCV`
(z wybraną przez siebie ilością iteracji) dla `XGBClassifier` 

Prównaj wyniki.